In [1]:
import sys
## To get helper functions from another folder
# sys.path.insert(0, '../') # if _helper_basics_ is in previous folder
now_i_am_at = 'dso' # home dso test
if now_i_am_at=='home': sys.path.insert(0, 'E:/Leonard HDD/Dropbox/DSO/Tasks/')
elif now_i_am_at=='dso': sys.path.insert(0, 'D:/Dropbox/DSO/Tasks')

from _helper_basics_ import *

%matplotlib inline
plt.rcParams['savefig.dpi'] = 100

%load_ext autoreload
%autoreload 2

print('keras', keras.__version__)
print('tensorflow', tf.__version__)

Using TensorFlow backend.


keras 2.0.6
tensorflow 1.2.1


In [12]:
from _helper_basics_ import *
%reload_ext autoreload

In [30]:
Archi_dir = ""
Weights_path = "Logs/"
if not os.path.exists(Weights_path): os.mkdir(Weights_path)
Ckpt_Mod_Weights_fold = Archi_dir+"Checkpoint_Model_Weights/"
if not os.path.exists(Ckpt_Mod_Weights_fold): os.mkdir(Ckpt_Mod_Weights_fold)

# Generate training data

In [2]:
def Addition_Data(input_classes ,sets_per_class ,num_classes):
    data_len = num_classes*sets_per_class

    y = np.repeat(np.arange(num_classes), sets_per_class).reshape(data_len).astype(np.float16)
    x1 = np.zeros((data_len,1)).astype(np.float16)
    x2 = np.zeros((data_len,1)).astype(np.float16)

    for idx in range(data_len):
        ## given y, chose random x1, then get the corresponding x2
        y_i = y[idx]
        if y_i == 0: 
            # x1,x2 already init as 0.
            continue
        elif y_i == (num_classes-1):
            x1[idx,0] = input_classes-1
            x2[idx,0] = input_classes-1
            continue

        x_i = np.round( np.random.rand(1)*(input_classes-1) )
        x_j = y_i-x_i
        count = 0
        while (x_j<0) or (x_j>(input_classes-1)):
            x_i = np.round( np.random.rand(1)*(input_classes-1) )
            x_j = y_i-x_i
            count += 1
        x1[idx,0] = x_i
        x2[idx,0] = x_j
        
    return x1,x2,y,data_len

In [3]:
## Training Set
input_classes = 100+1
sets_per_class = 500
num_classes = 200+1
x1,x2,y,data_len = Addition_Data(input_classes ,sets_per_class ,num_classes)
print('x1.shape',x1.shape)
print('x2.shape',x2.shape)
print('y.shape',y.shape)

x1.shape (100500, 1)
x2.shape (100500, 1)
y.shape (100500,)


## Split into Training Val Set

In [4]:
indices = np.arange(data_len)
np.random.shuffle(indices)
x1,x2,y = x1[indices],x2[indices],y[indices]
print('x1.shape',x1.shape)
print('x2.shape',x2.shape)
print('y.shape',y.shape)

x1.shape (100500, 1)
x2.shape (100500, 1)
y.shape (100500,)


In [5]:
x1_trg,x2_trg = x1[data_len//4:],x2[data_len//4:]
X_train = np.append(x1_trg,x2_trg,axis=1)
y_trg = y[data_len//4:]
Y_train = to_categorical(y_trg, num_classes=num_classes)
print('X_train.shape',X_train.shape)
print('Y_train.shape',Y_train.shape)

## Check for Error 
for idx in range(Y_train.shape[0]):
    if not X_train[idx][0]+X_train[idx][1] == np.argmax(Y_train[idx]):
        print('\nError idx              ', idx)
        print('X_train[idx]           ', X_train[idx][0]+X_train[idx][1], X_train[idx])
        print('y[0,idx]               ', y[0,idx])
        print('np.argmax(Y_train[idx])', np.argmax(Y_train[idx]))

X_train.shape (75375, 2)
Y_train.shape (75375, 201)


In [6]:
X_val = np.append(x1[:data_len//4],x2[:data_len//4],axis=1)
y_v = y[:data_len//4]
Y_val = to_categorical(y_v, num_classes=num_classes)
print('X_val.shape',X_val.shape)
print('Y_val.shape',Y_val.shape)

## Check for Error 
for idx in range(Y_val.shape[0]):
    if not X_val[idx][0]+X_val[idx][1] == np.argmax(Y_val[idx]):
        print('\nError idx              ', idx)
        print('X_val[idx]           ', X_val[idx][0]+X_val[idx][1], X_val[idx])
        print('y[0,idx]               ', y[0,idx])
        print('np.argmax(Y_val[idx])', np.argmax(Y_val[idx]))

X_val.shape (25125, 2)
Y_val.shape (25125, 201)


In [7]:
idx = 200
print('X_train[idx]           ', X_train[idx][0]+X_train[idx][1], X_train[idx])
print('np.argmax(Y_train[idx])',np.argmax(Y_train[idx]))

idx = 200
print('X_val[idx]           ', X_val[idx][0]+X_val[idx][1], X_val[idx])
print('np.argmax(Y_val[idx])',np.argmax(Y_val[idx]))

X_train[idx]            42.0 [  4.  38.]
np.argmax(Y_train[idx]) 42
X_val[idx]            136.0 [ 90.  46.]
np.argmax(Y_val[idx]) 136


In [68]:
pickle.dump(X_train, open('Addition_Data/X_train', 'wb'))
pickle.dump(Y_train, open('Addition_Data/Y_train', 'wb'))
pickle.dump(X_val, open('Addition_Data/X_val', 'wb'))
pickle.dump(Y_val, open('Addition_Data/Y_val', 'wb'))

X_train.shape (75375, 2)
Y_train.shape (75375, 201)
X_val.shape (25125, 2)
Y_val.shape (25125, 201)


In [6]:
X_train = pickle.load( open('X_train', 'rb'))
Y_train = pickle.load( open('Y_train', 'rb'))
X_val = pickle.load( open('X_val', 'rb'))
Y_val = pickle.load( open('Y_val', 'rb'))

In [7]:
print('X_train.shape',X_train.shape)
print('Y_train.shape',Y_train.shape)
print('X_val.shape',X_val.shape)
print('Y_val.shape',Y_val.shape)

X_train.shape (75375, 2)
Y_train.shape (75375, 201)
X_val.shape (25125, 2)
Y_val.shape (25125, 201)


### Remove numbers from the Training Data
Next step to remove prime numbers

In [8]:
## To remove a list of from the X_train
list_remove = [10,20,30,40,50,60,70,80,90]

X_train_removed = X_train+0.
Y_train_removed = Y_train+0.

for idx_remove in list_remove:
    indices_50 = list(np.where(X_train_removed[:,0]==idx_remove)[0]) + list(np.where(X_train_removed[:,1]==idx_remove)[0])
    print('len(indices_50)',len(indices_50))
    X_train_removed = np.delete(X_train_removed, indices_50 , axis=0)
    Y_train_removed = np.delete(Y_train_removed, indices_50 , axis=0)
print('\nX_train_50.shape',X_train_removed.shape)
print('Y_train_removed.shape',Y_train_removed.shape)
print('X_train.shape   ',X_train.shape)
print('Y_train.shape   ',Y_train.shape)

len(indices_50) 1766
len(indices_50) 1322
len(indices_50) 1134
len(indices_50) 982
len(indices_50) 966
len(indices_50) 1047
len(indices_50) 1110
len(indices_50) 1312
len(indices_50) 1663

X_train_50.shape (64144, 2)
Y_train_removed.shape (64144, 201)
X_train.shape    (75375, 2)
Y_train.shape    (75375, 201)


In [10]:
## Check for Error 
for idx_remove in list_remove:
    for idx in range(Y_train_removed.shape[0]):
        if X_train_removed[idx,0]==idx_remove or X_train_removed[idx,1]==idx_remove:
            print('\nError idx               ', idx)
            print('X_train_removed[idx]           ', X_train_removed[idx][0]+X_train_removed[idx][1], X_train_removed[idx])
            print('np.argmax(Y_train_removed[idx])', np.argmax(Y_train_removed[idx]))

idx = -1
print('X_train_removed[idx]           ', X_train_removed[idx,0]+X_train_removed[idx,1], X_train_removed[idx])
print('np.argmax(Y_train_removed[idx])',np.argmax(Y_train_removed[idx]))

X_train_removed[idx]            153.0 [ 89.  64.]
np.argmax(Y_train_removed[idx]) 153


# Saving

In [11]:
pickle.dump(X_train_removed, open('X_train_removed', 'wb'))
pickle.dump(Y_train_removed, open('Y_train_removed', 'wb'))

# pickle.dump(X_train_50, open('Addition_Data/X_train_tens_removed', 'wb'))
# pickle.dump(Y_train_50, open('Addition_Data/Y_train_tens_removed', 'wb'))

# Loading

In [12]:
X_train_removed = pickle.load( open('X_train_removed', 'rb'))
Y_train_removed = pickle.load( open('Y_train_removed', 'rb'))

In [14]:
print('X_train_removed.shape',X_train_removed.shape)
print('Y_train_removed.shape',Y_train_removed.shape)

X_train_removed.shape (64144, 2)
Y_train_removed.shape (64144, 201)
